In [128]:
import datetime
import pandas as pd

# 取得今天日期，並轉換為字串 'yyyymmdd' 格式
today = datetime.date.today().strftime('%Y%m%d')

# 設定檔案路徑，檔名為今天日期_TraderPosition.xlsx
file_path = f'data/{today}_TraderPosition.xlsx'

# 有哪些人
names = ['Jesse', 'Aaron', 'Mark1', 'Gary', 'Peggy', 'PETE']
# 要讀取的工作表名稱
sheet_names = names

# 將每個工作表的 DataFrame 存入以工作表名稱為 key 的字典中, 不讀取前兩列 & 只讀取A到I欄, 只擷取第一欄中前面兩字包含"US"或"TY"或"FV"或"TU"的row、但不包含total的列數
df_dict = {}
for sheet_name in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=2, usecols='A:I')
    df = df[df.iloc[:, 0].astype(str).str.slice(0, 2).isin(['US', 'TY', 'FV', 'TU']) & ~df.iloc[:, 0].astype(str).str.contains('total', case=False)]
    df_dict[sheet_name] = df

    # 將 DataFrame 存入變數 df_{sheet_name} 中
    var_name = f'df_{sheet_name}'
    globals()[var_name] = df

#print(df_dict)

In [129]:
import pandas as pd
import datetime

# 取得今天日期，並轉換為字串 'yymmdd' 格式
today = datetime.date.today().strftime('%y%m%d')

# 設定檔案路徑，檔名為海外債券一覽表{today}.xlsm
file_path = f'data/海外債券一覽表{today}.xlsm'

# 要讀取的工作表名稱
sheet_names = ['Jesse_UR', 'Jesse_Real', 'Aaron_UR', 'Aaron_Real', 'Mark1_UR', 'Mark1_Real',
               'Gary_UR', 'Gary_Real', 'Peggy_UR', 'Peggy_Real', 'PETE_UR', 'PETE_Real']

# 讀取Excel檔案中指定的工作表，並存入以工作表名稱為 key 的字典中
po_dict = {}
for sheet_name in sheet_names:
    po = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:I", header=0)
    start_row = po.loc[po.iloc[:, 0] == "USD_Futures"].index[0] + 1
    end_row = po.loc[po.iloc[:, 0] == "USD_total"].index[0]
    po = po.iloc[start_row:end_row, :]


    # 將 USD_Futures 那一列的資料轉換成對應的 column name
    column_names = list(po.iloc[0])
    po.columns = column_names
    po = po.iloc[1:, :]
    po = po.reset_index(drop=True)  # 重新編號 row index
    po_dict[sheet_name] = po

    # 將 DataFrame 存入變數 po_{sheet_name} 中
    var_name = f'po_{sheet_name}'
    globals()[var_name] = po

    po_dict[sheet_name].dropna(axis=0, how='all', inplace=True)

#print(po_dict)

# 創建一個空的 dataframe，列名為姓名，列數為0
for name in names:
    globals()[f'po_{name}_Real_New'] = pd.DataFrame(columns=po_dict[f'{name}_Real'].columns)

In [131]:
import pandas as pd

def process_conditions(df_name, po_name_UR, po_name_Real_New):
    condition_met = True
    while condition_met:
        condition_met = False
        for index, row in df_name.iterrows():
            product_id = row['ProductID']
            broker = row['Broker']
            matching_rows = po_name_UR[(po_name_UR['ProductID'] == product_id) & (po_name_UR['Broker'] == broker)]
            for ur_index, matching_row in matching_rows.iterrows():
                if matching_row['Lots'] != 0:
                    # 情況一
                    if row['Lots'] > 0 and matching_row['Lots'] < 0 and abs(row['Lots']) >= abs(matching_row['Lots']):
                        matching_row['MKT Price'] = row['Cost']  # 更新 MKT Price 欄位
                        po_name_Real_New = pd.concat([po_name_Real_New, pd.DataFrame([matching_row])], ignore_index=True)
                        df_name.at[index, 'Lots'] = row['Lots'] - abs(matching_row['Lots'])  # 更新 df_人名 的 Lots 欄位
                        po_name_UR.at[ur_index, 'Lots'] = 0  # 更新 po_人名_UR 的 Lots 欄位
                        condition_met = True
                        break
                    # 情況二
                    elif row['Lots'] > 0 and matching_row['Lots'] < 0 and abs(row['Lots']) < abs(matching_row['Lots']):
                        new_row = matching_row.copy()
                        new_row['Lots'] = -row['Lots']  # 更新 Lots 欄位
                        new_row['MKT Price'] = row['Cost']  # 更新 MKT Price 欄位
                        po_name_Real_New = pd.concat([po_name_Real_New, pd.DataFrame([matching_row])], ignore_index=True)
                        po_name_UR.at[ur_index, 'Lots'] = matching_row['Lots'] + row['Lots']  # 更新 po_人名_UR 的 Lots 欄位
                        df_name.at[index, 'Lots'] = 0  # 更新 df_人名 的 Lots 欄位
                        condition_met = True
                        break
                    # 情況三
                    elif row['Lots'] < 0 and matching_row['Lots'] > 0 and abs(row['Lots']) >= abs(matching_row['Lots']):
                        matching_row['MKT Price'] = row['Cost']  # 更新 MKT Price 欄位
                        po_name_Real_New = pd.concat([po_name_Real_New, pd.DataFrame([matching_row])], ignore_index=True)
                        df_name.at[index, 'Lots'] = row['Lots'] + matching_row['Lots']  # 更新 df_人名 的 Lots 欄位
                        po_name_UR.at[ur_index, 'Lots'] = 0  # 更新 po_人名_UR 的 Lots 欄位
                        condition_met = True
                        break
                     # 情況四
                    elif row['Lots'] < 0 and matching_row['Lots'] > 0 and abs(row['Lots']) < abs(matching_row['Lots']):
                        new_row = matching_row.copy()
                        new_row['Lots'] = -row['Lots']  # 更新 Lots 欄位
                        new_row['MKT Price'] = row['Cost']  # 更新 MKT Price 欄位
                        po_name_Real_New = pd.concat([po_name_Real_New, pd.DataFrame([matching_row])], ignore_index=True)
                        po_name_UR.at[ur_index, 'Lots'] = matching_row['Lots'] + row['Lots']
                        po_name_UR.at[ur_index, 'Lots'] = matching_row['Lots'] + row['Lots']  # 更新 po_人名_UR 的 Lots 欄位
                        df_name.at[index, 'Lots'] = 0  # 更新 df_人名 的 Lots 欄位
                        condition_met = True
                        break
            if not condition_met:
                # 情況五
                for index, row in df_name.iterrows():
                    if row['Lots'] != 0:
                        matching_rows = po_name_UR[(po_name_UR['Lots'] == 0)]
                        for ur_index, matching_row in matching_rows.iterrows():
                            po_name_UR.loc[ur_index] = row  # 使用 .loc 來更新 po_人名_UR 的整行資料
                            df_name.at[index, 'Lots'] = 0  # 更新 df_人名 的 Lots 欄位
                            break  # 當找到匹配的行並完成更新後，跳出迴圈

    return df_name, po_name_UR, po_name_Real_New


for name in names:
    df_name = globals()[f'df_{name}']
    po_name_UR = globals()[f'po_{name}_UR']
    po_name_Real_New = globals()[f'po_{name}_Real_New']

    # 大前提和所有情況
    df_name, po_name_UR, po_name_Real_New = process_conditions(df_name, po_name_UR, po_name_Real_New)

    globals()[f'po_{name}_Real_New'] = po_name_Real_New

In [132]:
po_Jesse_UR

,ProductID,Trade Date,closed date,closed ticket no.,Lots,Broker,DV01,Ticket Number,Cost
0,TYM3,2023/04/04,NaN,NaN,-50,CAPITAL,0,2023040400007,115.34375
1,TYM3,2023/04/04,NaN,NaN,-50,CAPITAL,0,2023040400010,115.15625
2,TYM3,2023/04/04,NaN,NaN,100,CAPITAL,0,2023040400012,115.21875
3,TYM3,2023/03/08,NaN,NaN,0,CAPITAL,0,2023030800037,111.234375
4,TYM3,2023/03/10,NaN,NaN,0,CAPITAL,0,2023031000059,112.578125
5,TYH3,2020/03/09,NaN,NaN,0,CAPITAL,0,2020030900083,138.875
6,TYH3,2020/03/09,NaN,NaN,0,CAPITAL,0,2020030900084,139.0625
7,TYH3,2020/05/14,NaN,NaN,0,CAPITAL,0,2020051400006,139.5
8,TYH3,2020/05/13,NaN,NaN,0,CAPITAL,0,2020051300004,139.3125
9,TYH3,2020/11/04,NaN,NaN,0,CAPITAL,0,2020110400018,138.90625


In [133]:
po_Mark1_Real_New

,ProductID,Trade Date,closed date,closed ticket no.,Lots,Broker,DV01,Ticket Number,Cost,MKT Price
0,FVM3,2023/03/29,NaN,NaN,10,CAPITAL,-427,2023032900002,109.273438,109.390625
1,FVM3,2023/03/29,NaN,NaN,10,CAPITAL,-427,2023032900076,109.375000,109.382812


In [136]:
#上面是對從df_人名到po_人名_UR以及po_人名_Real_New的code
#這個是對po_人名_UR自己的code

def process_conditions_v3(df_name, po_name_UR, po_name_Real_New):
    condition_met = True
    while condition_met:
        condition_met = False
        for index, row1 in po_name_UR.iterrows():
            if row1['Lots'] != 0:
                product_id = row1['ProductID']
                broker = row1['Broker']
                matching_rows = po_name_UR[(po_name_UR['ProductID'] == product_id) & (po_name_UR['Broker'] == broker)]
                for ur_index, row2 in matching_rows.iterrows():
                    if row2['Lots'] != 0:
                        # 情況一
                        if row1['Lots'] > 0 and row2['Lots'] < 0 and abs(row1['Lots']) >= abs(row2['Lots']):
                            row3 = row2.copy()
                            row3['MKT Price'] = row1['Cost']  # 更新 MKT Price 欄位
                            po_name_Real_New = pd.concat([po_name_Real_New, row3.to_frame().T], ignore_index=True)
                            po_name_UR.at[index, 'Lots'] = row1['Lots'] - abs(row2['Lots'])  # 更新 po_人名_UR 的 Lots 欄位
                            po_name_UR.at[ur_index, 'Lots'] = 0  # 更新 row2 的 Lots 欄位
                            condition_met = True
                            break
                        # 情況二
                        elif row1['Lots'] > 0 and row2['Lots'] < 0 and abs(row1['Lots']) < abs(row2['Lots']):
                            row3 = row2.copy()
                            row3['Lots'] = -row1['Lots']  # 更新 Lots 欄位
                            row3['MKT Price'] = row1['Cost']  # 更新 MKT Price 欄位
                            po_name_Real_New = pd.concat([po_name_Real_New, row3.to_frame().T], ignore_index=True)
                            po_name_UR.at[ur_index, 'Lots'] = row2['Lots'] + row1['Lots']  # 更新 row2 的 Lots 欄位
                            po_name_UR.at[index, 'Lots'] = 0  # 更新 row1 的 Lots 欄位
                            condition_met = True
                            break
                        # 情況三
                        elif row1['Lots'] < 0 and row2['Lots'] > 0 and abs(row1['Lots']) >= abs(row2['Lots']):
                            row3 = row2.copy()
                            row3['MKT Price'] = row1['Cost']  # 更新 MKT Price 欄位
                            po_name_Real_New = pd.concat([po_name_Real_New, row3.to_frame().T], ignore_index=True)
                            po_name_UR.at[index, 'Lots'] = row1['Lots'] + row2['Lots']  # 更新 row1 的 Lots 欄位
                            po_name_UR.at[ur_index, 'Lots'] = 0  # 更新 row2 的 Lots 欄位
                            condition_met = True
                            break
                        # 情況四
                        elif row1['Lots'] < 0 and row2['Lots'] > 0 and abs(row1['Lots']) < abs(row2['Lots']):
                            row3 = row2.copy()
                            row3['Lots'] = -row1['Lots']  # 更新 Lots 欄位
                            row3['MKT Price'] = row1['Cost']  # 更新 MKT Price 欄位
                            po_name_Real_New = pd.concat([po_name_Real_New, row3.to_frame().T], ignore_index=True)
                            po_name_UR.at[ur_index, 'Lots'] = row2['Lots'] + row1['Lots']  # 更新 row2 的 Lots 欄位
                            po_name_UR.at[index, 'Lots'] = 0  # 更新 row1 的 Lots 欄位
                            condition_met = True
                            break
    return po_name_UR, po_name_Real_New


for name in names:
    po_name_UR = globals()[f'po_{name}_UR']
    po_name_Real_New = globals()[f'po_{name}_Real_New']

    # 執行新的大前提和情況
    po_name_UR, po_name_Real_New = process_conditions_v2(df_name, po_name_UR, po_name_Real_New)

    globals()[f'po_{name}_Real_New'] = po_name_Real_New

    
#將po_UR的DV01調整
def update_dv01(row):
    product_id = row['ProductID']
    lots = row['Lots']
    if product_id.startswith('TU'):
        return -18.6 * 2 * lots
    elif product_id.startswith('FV'):
        return -42.7 * lots
    elif product_id.startswith('TY'):
        return -77 * lots
    elif product_id.startswith('US'):
        return -198 * lots
    else:
        return row['DV01']

    
for name in names:
    po_name_Real_New = globals()[f'po_{name}_Real_New']
    po_name_Real_New['MTM P&L'] = po_name_Real_New.apply(calculate_mtm_pnl, axis=1)
    globals()[f'po_{name}_Real_New'] = po_name_Real_New


for name in names:
    po_name_UR = globals()[f'po_{name}_UR']
    po_name_UR['DV01'] = po_name_UR.apply(update_dv01, axis=1)
    globals()[f'po_{name}_UR'] = po_name_UR


#將po_Real_New新增欄位MTM P&L，並用公式計算
def calculate_mtm_pnl(row):
    if row['ProductID'][:2] == "TU":
        return (row['MKT Price'] - row['Cost']) * row['Lots'] * 2000
    else:
        return (row['MKT Price'] - row['Cost']) * row['Lots'] * 1000
    
# 將所有 DataFrame 寫入一個新的 Excel 檔案中，並將df_人名工作表放在所有其他工作表之後。
output_file_path = f'data/{today}_TraderPosition_Output.xlsx'

with pd.ExcelWriter(output_file_path) as writer:
    for name in names:
        po_name_UR = globals()[f'po_{name}_UR']
        po_name_Real_New = globals()[f'po_{name}_Real_New']

        po_name_UR.to_excel(writer, sheet_name=f'po_{name}_UR', index=False)
        po_name_Real_New.to_excel(writer, sheet_name=f'po_{name}_Real_New', index=False)

    for name in names:
        df_name = globals()[f'df_{name}']
        df_name.to_excel(writer, sheet_name=f'df_{name}', index=False)

In [119]:
po_PETE_UR

,ProductID,Trade Date,closed date,closed ticket no.,Lots,Broker,DV01,Ticket Number,Cost
0,TYH3,2023/02/15,NaN,NaN,0,CAPITAL,0.0,2023021500076,112.109375
1,TYH3,2023/02/15,NaN,NaN,0,CAPITAL,0.0,2023021500077,112.109375
2,TYM3,2023/02/21,NaN,NaN,0,CAPITAL,0.0,2023022100008,112.28125
3,TYM3,2023/03/02,NaN,NaN,0,CAPITAL,0.0,2023030200002,110.9375
4,TYM3,2023/03/02,NaN,NaN,0,CAPITAL,0.0,2023030200007,110.75
5,TYM3,2023/03/01,NaN,NaN,0,CAPITAL,0.0,2023030100004,111.375
6,TYM3,2023/03/01,NaN,NaN,0,CAPITAL,0.0,2023030100009,111.3125
7,TUM3,2023/03/02,NaN,NaN,0,CAPITAL,-0.0,2023030200003,101.59375
8,TUM3,2023/03/02,NaN,NaN,0,CAPITAL,-0.0,2023030200008,101.625
9,TUH3,2023/02/02,NaN,NaN,0,CAPITAL,-0.0,2023020200060,103.109375


In [120]:
po_PETE_Real_New

,ProductID,Trade Date,closed date,closed ticket no.,Lots,Broker,DV01,Ticket Number,Cost,MKT Price,MTM P&L
0,TYH3,2023/02/15,NaN,NaN,-10,CAPITAL,0,2023021500077,112.109375,112.109375,-0.000
1,TYM3,2023/03/02,NaN,NaN,-10,CAPITAL,0,2023030200007,110.75,112.28125,-15312.500
2,TYM3,2023/03/02,NaN,NaN,-10,CAPITAL,0,2023030200007,110.75,110.9375,-1875.000
3,TYM3,2023/03/01,NaN,NaN,-10,CAPITAL,0,2023030100004,111.375,111.3125,625.000
4,TUH3,2023/02/02,NaN,NaN,-10,CAPITAL,0,2023020200063,103.097656,103.109375,-234.375
5,TUH3,2023/02/02,NaN,NaN,-10,CAPITAL,0,2023020200065,103.089844,103.105469,-312.500
6,TUH3,2023/02/02,NaN,NaN,-10,CAPITAL,0,2023020200065,103.089844,103.101562,-234.375
7,TUH3,2023/02/02,NaN,NaN,-10,CAPITAL,0,2023020200065,103.089844,103.09375,-78.125
8,TUH3,2023/02/02,NaN,NaN,-30,CAPITAL,0,2023020200066,103.085938,103.070312,937.500
9,TUH3,2023/02/02,NaN,NaN,-10,CAPITAL,0,2023020200067,103.082031,103.0625,390.625
